# QUT MDM Profiling

This document details the code and techniques used in the strategic profiling exercises during the construction of the QUT MDM system. 

<br>

---



## Introduction

### Associated Files

| **File**  | **Description**  |
|---|---|
|  `handover.ipynb` | The primary document is entitled `handover.ipynb`. This file contains the desription of the process, as well as the code which performs the analysis and presents the results.  |
|  `requirements.txt` | The `requirements.txt` file details the Python libraries which are required to run the analysis. For information on how to set up a working environment see the **Installation** section below.  |
|  `handover.html` |  The `handover.html` file is a static pdf copy of the `handover.ipynb` file, created for ease of reading outside of a coding environment. It contains scrollable codebocks. |
|  `handover.pdf` |  The `handover.pdf` file is a static pdf copy of the `handover.ipynb` file, created for ease of annotation outside of a coding environment. |

<br>



### Installation
The following is a quick how-to guide for setting up and running this analysis on your machine. If you are already equipped to run data analysis using Python and Jupyter, feel free to skip ahead.

#### Pre-requisites:
- This analysis was conducted using the `Python` programming language, using the `Jupyter Notebook` REPL platform. To open and manipulate the `main.ipynb` file, a code editor is required. We recommend [`VS Code`](https://code.visualstudio.com/download), running the [`Jupyter`](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) extension. 

- To run the code, `Python 3.10+` is recommended. Instructions on installing Python for various operating systems can be found [here](https://wsvincent.com/install-python/).

- You will also need [`pip`](https://pip.pypa.io/en/stable/installation/) in order to install dependencies.

#### Creating a virtual environment:

1. Begin by installing `virtualenv` globally on your machine. You can do this by  the command ```python3 -m pip install virtualenv``` in your terminal.
1. In the terminal, navigate into the directory containing the analysis files.
1. Next, use `virtualenv` to create a virtual environment in which to run the analysis. The command is ```python3 -m virtualenv venv```
1. Activate the virual environment. On Unix systems the command is ```source ./venv/bin/activate```. On Windows you should use ```.\venv\Scripts\activate``` (you'll need to perform this step every time you want to work with the file.)
1. Install the project dependencies by running ```python3 -m pip install -r requirements.txt```
1. *That's it, you are now ready to run the file!*

<br>



### Input Data:
Since this document and it's associated files have been prepared for dissemination to various internal stakeholders, the input data which were used to generate the analysis have not been included in the file tree. A description of the input data can be found below in the. To reproduce of extend this analysis, data will need to be supplied of the same or similar format to the original inputs.

<br>

---

## Profiling
The code for profiling and the results are set out below with annotations. Depending on the inputs and goals, modification/adaptation may be required if you want to repeat/extend the analysis.

### Importing Dependencies:

In [ ]:
import numpy as np
import pandas as pd
import re
import phonenumbers
import datetime
import warnings

from typing import Optional
from validate_email import validate_email as _validate_email
from IPython.display import display
from pandas_profiling import ProfileReport
from collections import defaultdict


### Settings

This section contains code that defines settings for the rest of the profiling exercise. Making modifications here is the simplest and safest way of tweaking the analysis.

In [ ]:
# This dictionary defines the locations of the input .csv files
# Note that the files listed here are not included in the distributed version of this handover
# To run this analysis, you will need to obtain data extracts containing the same/similar information
# It may be necessary to modify code throughout this file to suit your inputs
INPUT_DATA_ADDRESSES = {
    "ascender_employees": "./source_data/employee_grab.csv",
    "dep_prospects": "./source_data/dep_grab.csv",
    "studylink_applicants": "./source_data/studylink_grab.csv",
    "sams_students": "./source_data/sams_student_grab.csv",
    "sams_applicants": "./source_data/sams_applicant_grab.csv"
}

# This dictionary records the fields in each data set that we will use as "primary key"-style identifiers 
PK_FIELDS = {
    "ascender_employees": "EMPLOYEE_ID",
    "dep_prospects": "CONTACT_ID",
    "studylink_applicants": "APPLICANT_ID",
    "sams_students": "QUT_GUID",
    "sams_applicants": "QUT_GUID"
}

# A list of the pairs of columns on which we will detect potential duplicates
DUPLICATE_CRITERIA = [
    ("PHONE_MOBILE", "GIVEN_NAME"),
    ("PHONE_MOBILE", "DOB"),
    ("PERSONAL_EMAIL", "GIVEN_NAME"),
    ("PERSONAL_EMAIL", "DOB")
]

# A dict of the optimal ways to align linked entries in datasets
MATCHFIELDS = {
    "QUT_GUID": [
        ("dep_prospects", "ascender_employees"),
        ("ascender_employees", "sams_applicants"),
        ("ascender_employees", "sams_students"),
        ("sams_students", "sams_applicants"),
    ],
    "STUDENT_ID": [
        ("sams_students", "studylink_applicants"),
        ("studylink_applicants", "sams_applicants")
    ]
}

# This dict governs how to choose a value for a field when different sources have different values for that field for a single person
# Note that a valid value always trumps an invalid value, and a non-null value always trumps a null value
FIELD_SYSTEM_PRECEDENCES = {
    'ADDR1_HOME': ["ascender_employees", "sams_applicants", "sams_students", "studylink_applicants", "dep_prospects"],
    'ADDR2_HOME': ["ascender_employees", "sams_applicants", "sams_students", "studylink_applicants", "dep_prospects"],
    'ADDR3_HOME': ["ascender_employees", "sams_applicants", "sams_students", "studylink_applicants", "dep_prospects"],
    'CNTRY_HOME': ["ascender_employees", "sams_applicants", "sams_students", "studylink_applicants", "dep_prospects"],
    'DOB': ["sams_applicants", "ascender_employees", "sams_students", "studylink_applicants", "dep_prospects"],
    'FAMILY_NAME': ["sams_applicants", "ascender_employees", "sams_students", "studylink_applicants", "dep_prospects"],
    'FULL_NAME': ["sams_applicants", "ascender_employees", "sams_students", "studylink_applicants", "dep_prospects"],
    'GENDER': ["sams_applicants", "ascender_employees", "sams_students", "studylink_applicants", "dep_prospects"],
    'GIVEN_NAME': ["sams_applicants", "ascender_employees", "sams_students", "studylink_applicants", "dep_prospects"],
    'OTHER_NAME': ["sams_applicants", "ascender_employees", "sams_students", "studylink_applicants", "dep_prospects"],
    'PERSONAL_EMAIL': ["ascender_employees", "sams_applicants", "sams_students", "studylink_applicants", "dep_prospects"],
    'PHONE_MOBILE': ["sams_applicants", "studylink_applicants", "sams_students", "ascender_employees", "dep_prospects"],
    'POSTC_HOME': ["ascender_employees", "sams_applicants", "sams_students", "studylink_applicants", "dep_prospects"],
    'PREFERRED_NAME': ["sams_applicants", "ascender_employees", "sams_students", "studylink_applicants", "dep_prospects"],
    'SALUTATION': ["sams_applicants", "ascender_employees", "studylink_applicants", "sams_students", "dep_prospects"],
    'STATE_HOME': ["ascender_employees", "sams_applicants", "sams_students", "studylink_applicants", "dep_prospects"],
    'SUBRB_HOME': ["ascender_employees", "sams_applicants", "sams_students", "studylink_applicants", "dep_prospects"]
}

### Reading Data
This section reads data from the files specified in the `INPUT_DATA_ADDRESSES` setttings variable.

In [ ]:
# We read each of the files into a value in a dictionary.
input_data = {dataset_name: pd.read_csv(file_location, dtype=str) for dataset_name, file_location in INPUT_DATA_ADDRESSES.items()}

### Generating Exploratory Reports

This section creates the exploratory reports. Simply uncomment and run the code.

In [ ]:
# for dataset_name, dataset in input_data.items():
#    report = ProfileReport(dataset, minimal=True, vars={"cat": {"n_obs": 20}})
#    report.to_file(f"./exploratory_profiles/{dataset_name}_profile.html")

### Completeness:
The following block of code calculates the completeness of each column in each dataset, and displays these values in a series of convenient tables.

In [ ]:
# We iterate through each dataset
for dataset_name, dataset in input_data.items():

    # First we calculate how long the dataset is
    dataset_length = len(dataset)
    
    # Then we create a temporary dataframe to hold our calculations
    to_display = pd.DataFrame({
        # The first column of the temp dataframe contains the names of each column in the dataset
        "column": [column for column in dataset.columns],
        # The second column shows how many non-null values there are for each column
        "count": [len(dataset[dataset[column].notna()]) for column in dataset.columns],
        # The third column calculates the relative frequency of non-null values 
        "completeness": [len(dataset[dataset[column].notna()])/dataset_length for column in dataset.columns],
        # The fourth column gives us an example value from the column
        "sample": [dataset[dataset[column].notna()][column].mode()[0] if len(dataset[dataset[column].notna()][column])>0 else np.nan for column in dataset]
    # we apply a little styling to our temp dataframe so that the completeness column is displayed as a percentage
    }).style.format({
        "completeness": '{:,.2%}'.format
    })

    # A quick string output to show what data source we're talking about and how big it is
    print(f"\n\n{dataset_name} ({dataset_length} records):\n".upper())
    # This display() function is the one we imported from Ipython up the top - it just improves readability
    display(to_display)

    print("\n-------------------------------------------------------------")


### Validity

Calculating validity is complex. First let's define some functions to automate the work for us...

In [ ]:
# We define an alpha regex to check that a string contains only acceptable characters for names
ALPHA_REGEX = re.compile("^[A-Za-zÀ-ÖØ-öø-ÿ -./']*$")

# A function to validate names.
def validate_name(name_string, banned_values=[], polluting_values=[]):
    """Validates a value by checking that it is an acceptable name.
    
    Checks that the value is a string containing only alphabetical characters 
    (including diacritics), plus spaces, fullstops, dashes, apostophes, and 
    forward-slashes. Currently fails the empty string "" as invalid.
    
    Args:
        name_string: Any value that is potentially a name. 
        banned_values: A list of specific values which are invalid.
        polluting_values: A list of values which cannot appear as substrings.

    Returns: 
        A boolean.
    """
    return (isinstance(name_string, str) 
            and bool(ALPHA_REGEX.match(name_string)) 
            and not any([name_string==value for value in banned_values]) 
            and not any([value in name_string for value in polluting_values]))

# A function to validate birthdates.
def validate_birthdate(date_string, date_format, banned_values=[], earliest_date=None, latest_date=None):
    """Validates a value by checking that it is an acceptable birthdate.

    Attempts to apply the datetime.strptime method to the value, returning 
    False on failure. Then rules out common "default" birthdates, i.e., 
    "01/01/1901".

    Args:
        date_string: Any value that is potentially a birthdate.
        date_format: A time format conforming to the 1998 C standard, example:
            '%Y-%m-%d', '%d/%m/%Y', etc...
        banned_values: A list of specific values which are invalid.
        earliest_date: The earliest allowable birthday
        latest_date: The latest allowable birthday
    
    Returns: 
        A boolean.
    """
    try:
        grokked_date = datetime.datetime.strptime(date_string, date_format)
        if earliest_date and grokked_date < earliest_date:
            return False
        if latest_date and grokked_date > latest_date:
            return False
    except (ValueError, TypeError):
        return False
    return not date_string in banned_values

# A function to validate email addresses
def validate_email(email_string, banned_values=[], polluting_values=["qut.edu.au"] ):
    """Validates a value by checking that it is an acceptable email address.

    Checks format with regex and returns False for blacklisted domains 
    (i.e., common throwaway addresses). For individual address validation
    it is possible to run _validat_email with check_dns and check_smtp set
    to True, however this is not practical for large datasets.

    Args:
        email_string: Any value that is potentially an email address.
        banned_values: A list of specific values which are invalid.
        polluting_values: A list of values which cannot appear as substrings.

    Returns: 
        A boolean.
    """
    return (isinstance(email_string, str) 
            and _validate_email(
                email_address=email_string, 
                check_format=True, 
                check_blacklist=True, 
                check_dns=False, 
                check_smtp=False) 
            and not any([email_string==value for value in banned_values]) 
            and not any([value in email_string for value in polluting_values]))

# A function to prepare mobile phone numbers for validation
def _default_mobile_value_treatment(mobile_value):
    """Attempts to standardise a mobile string prior to validation.

    This is a good candidate for a customisation, depending on your data.
    Process:
        - All non-numerical characters are removed.
        - Nine-digit numbers are assumed to be Australian mobiles with the 
            leading 0 omitted; the 0 is prepended. 
        - 10 digit numbers beginning with "04" are assumed to be Australian 
            mobiles; these are parsed as-is. 
        - All other numbers are assumed to be international numbers, and are
            prepended with "+".
    
    Args: 
        mobile_value: the value to treat.

    Returns:
        A treated string representing a mobile phone number.
    """

    try:
        mobile_string = str(int(float(mobile_value)))
    except ValueError:
        mobile_string = str(mobile_value)

    mobile_string=re.sub('[^0-9]','', mobile_string)


    if len(mobile_string)==9:
        mobile_string = f"0{mobile_string}"
    elif not (len(mobile_string)==10 and mobile_string[0:2]=="04"):
        mobile_string=f"+{mobile_string}"
    
    return mobile_string

# A function to validate mobile phone numbers
def validate_mobile_string(mobile_value, treatment=_default_mobile_value_treatment, banned_values=[]):
    """Validates a mobile string by attempting to parse it.
    
    Uses Google's LibPhoneNumbers parser.

    Args:
        mobile_value: Any value that is potentially a mobile phone number.
        treatment: A function to standardise values. You should probably 
            override the default here, especially if you haven't read that
            function. Failing all else just use `lambda x: x` to validate the
            value as-is.
        banned_values: A list of specific values which are invalid.
        polluting_values: A list of values which cannot appear as substrings.

    Returns: 
        A boolean.    
    """
    if any([mobile_value==value for value in banned_values]):
        return False
    try:
        parsed = phonenumbers.parse(treatment(mobile_value), region="AU")
    except phonenumbers.NumberParseException:
        return False
    return phonenumbers.is_valid_number(parsed)

# A handy dictionary to contain all of our validation functions
# We wrap the function in a lambda, inserting the arguments we want to use for each field.
validation_dict = {
    "DOB": lambda value: validate_birthdate(value, date_format='%Y-%m-%d', earliest_date=datetime.datetime(1922, 1,1), latest_date=datetime.datetime(2008, 1, 1)),
    "PERSONAL_EMAIL": lambda value: np.nan if value is np.nan else validate_email(value),
    "SALUTATION": lambda value: np.nan if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "GIVEN_NAME": lambda value: np.nan if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "OTHER_NAME": lambda value: np.nan if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "FAMILY_NAME": lambda value: np.nan if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "FULL_NAME": lambda value: np.nan if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "PREFERRED_NAME": lambda value: np.nan if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "PHONE_MOBILE": lambda value: np.nan if value is np.nan else validate_mobile_string(value),
    "GENDER": lambda value: np.nan if value is np.nan else value in ["F", "M", "X"],
}

Now we can get down to business...

In [ ]:
# The code we're about to run generates some irrelevant warnings, so let's catch them
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=UserWarning)

    # We iterate through each dataset
    for dataset_name, dataset in input_data.items():

        # first, we need one temp frame to hold the results of our validations
        validity_frame = pd.DataFrame()

        # our validity_frame mirrors the
        for column in dataset.columns:
            if column in validation_dict:
                validity_frame[column] = dataset[column].apply(validation_dict[column])    

        to_display = pd.DataFrame({
            # The first column of the temp dataframe contains the names of each column in the dataset
            "column": [column for column in validity_frame.columns],
            # The second column shows how many non-null values there are for each column
            "count": [len(dataset[dataset[column].notna()]) for column in validity_frame.columns],
            # The third column calculates how many valid values there are in each column 
            "valid": [len(validity_frame[validity_frame[column].notna()][validity_frame[column]==True]) for column in validity_frame.columns],
        
        })

        # Finally let's add another column to show validity as a percentage
        to_display["validity"] = to_display["valid"]/to_display["count"]

        print(f"\n\n{dataset_name.upper()}")
        display(to_display.style.format({
            # we apply a little styling to our temp dataframe so that the validity column is displayed as a percentage
            "validity": '{:,.2%}'.format
        }))
        print("\n-------------------------------------------------------------")


Turns out we are going to quarantine records that have invalid field values though. Let's look at the impact of that. First, what happens if we allow null values but disallow invalid ones?

In [ ]:
# A function to prepare mobile phone numbers for validation
def _default_mobile_value_treatment(mobile_value):
    """Attempts to standardise a mobile string prior to validation.

    This is a good candidate for a customisation, depending on your data.
    Process:
        - All non-numerical characters are removed.
        - Nine-digit numbers are assumed to be Australian mobiles with the 
            leading 0 omitted; the 0 is prepended. 
        - 10 digit numbers beginning with "04" are assumed to be Australian 
            mobiles; these are parsed as-is. 
        - All other numbers are assumed to be international numbers, and are
            prepended with "+".
    
    Args: 
        mobile_value: the value to treat.

    Returns:
        A treated string representing a mobile phone number.
    """

    try:
        mobile_string = str(int(float(mobile_value)))
    except ValueError:
        mobile_string = str(mobile_value)

    mobile_string=re.sub('[^0-9+]','', mobile_string)

    if mobile_string[:4] == "6161":
        mobile_string=f"+{mobile_string[2:]}"
    elif mobile_string[:5] == "+6161":
        mobile_string=f"+{mobile_string[3:]}"

    if len(mobile_string) == 8:
        mobile_string=f"+617{mobile_string}"

    if len(mobile_string)==9:
        mobile_string = f"+61{mobile_string}"
    
    if (len(mobile_string)==10 and mobile_string[0:2] in ["04", "02", "03", "07", "08"]):
        mobile_string=f"+61{mobile_string[1:]}"
    elif len(mobile_string) >= 10 and not mobile_string[0] in ["+", "00"]:
        mobile_string=f"+{mobile_string}"
    


    return mobile_string

# A function to validate mobile phone numbers
def validate_mobile_string(mobile_value, treatment=_default_mobile_value_treatment, banned_values=[], try_region="AU"):
    """Validates a mobile string by attempting to parse it.
    
    Uses Google's LibPhoneNumbers parser.

    Args:
        mobile_value: Any value that is potentially a mobile phone number.
        treatment: A function to standardise values. You should probably 
            override the default here, especially if you haven't read that
            function. Failing all else just use `lambda x: x` to validate the
            value as-is.
        banned_values: A list of specific values which are invalid.
        polluting_values: A list of values which cannot appear as substrings.

    Returns: 
        A boolean.    
    """
    if any([mobile_value==value for value in banned_values]):
        return False
    try:
        parsed = phonenumbers.parse(treatment(mobile_value), region=try_region)
    except phonenumbers.NumberParseException:
        return False
    return phonenumbers.is_valid_number(parsed)

def convert_to_cldr(region):
    match region:
        case "CHINA":
            return "CN"
        case "INDIA":
            return "IN"
        case "HONG KONG":
            return "HK"
        case "PHILIPPINES (THE)":
            return "PH"
        case "VIET NAM":
            return "VN"
        case "VIETNAM":
            return "VN"
        case "SAUDI ARABIA":
            return "SA"
        case "THAILAND":
            return "TH"
        case "SRI LANKA":
            return "LK"
        case "BANGLADESH":
            return "BD"
        case "GERMANY":
            return "DE"
        case "CAMEROON":
            return "CM"
        case "SOUTH AFRICA":
            return "ZA"
        case "MALAYSIA":
            return "MY"
        case "UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND":
            return "GB"
        case "ICELAND":
            return "IS"
        case "KAZAKHSTAN":
            return "KZ"
        case "FRANCE":
            return "FR"
        case "UNITED ARAB EMIRATES (THE)":
            return "AE"
        case "PAKISTAN":
            return "PK"
        case "BRAZIL":
            return "BR"
        case "PAPUA NEW GUINEA":
            return "PG"
        case "IRAN (ISLAMIC REPUBLIC OF)":
            return "IR"
        case "TAIWAN (PROVINCE OF CHINA)":
            return "TW"
        case "SINGAPORE":
            return "SG"
        case "JAPAN":
            return "JP"
        case "KOREA (THE REPUBLIC OF)":
            return "KR"
        case "ETHIOPIA":
            return "ET"
        case "CANADA":
            return "CA"
        case "MACAO":
            return "MO"
        case "MALAYSIA":
            return "MY"
        
        case other:
            return region

def try_and_validate(row):
    mynum = row["PHONE_MOBILE"]
    if mynum[:4] == "0011" or mynum[:5] == "+0011":
        mynum = f"+{mynum[4:]}"
    if mynum[:2] == "00" or mynum[:3] == "+00":
        mynum = f"+{mynum[2:]}"
    return validate_mobile_string(mynum, treatment=lambda x: x, try_region=convert_to_cldr(str(row["CNTRY_HOME"])))

validation_dict = {
    "DOB": lambda value: True if value is np.nan else validate_birthdate(value, date_format='%Y-%m-%d', earliest_date=datetime.datetime(1922, 1,1), latest_date=datetime.datetime(2008, 1, 1)),
    "PERSONAL_EMAIL": lambda value: True if value is np.nan else validate_email(value, polluting_values=[]),
    "SALUTATION": lambda value: True if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "GIVEN_NAME": lambda value: True if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "OTHER_NAME": lambda value: True if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "FAMILY_NAME": lambda value: True if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "FULL_NAME": lambda value: True if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "PREFERRED_NAME": lambda value: True if value is np.nan else validate_name(value, banned_values=["", ".", "-"], polluting_values=["Unknown", "UNKNOWN", "unknown", "DUPLICATE", "Duplicate", "duplicate"]),
    "PHONE_MOBILE": lambda value: True if value is np.nan else validate_mobile_string(value),
}

# The code we're about to run generates some irrelevant warnings, so let's catch them
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=UserWarning)

    # We iterate through each dataset
    for dataset_name, dataset in input_data.items():
        print(dataset_name.upper())
        # first, we need one temp frame to hold the results of our validations
        validity_frame = dataset.copy(deep=True)

        if not "CNTRY_HOME" in dataset.columns:
            bad = validity_frame[~validity_frame["PHONE_MOBILE"].apply(validation_dict["PHONE_MOBILE"])][["PHONE_MOBILE"]]
            print(f"\ndropped {len(bad)} of {len(dataset)} ({'{:.2%}'.format(len(bad)/len(dataset))})")
            display(bad.head(5))
        else:
            bad = validity_frame[~validity_frame["PHONE_MOBILE"].apply(validation_dict["PHONE_MOBILE"])][["PHONE_MOBILE", "CNTRY_HOME"]]
            bad["fixable with country code"] = bad.apply(try_and_validate, axis=1)
            print(f"\ndropped {len(bad[~bad['fixable with country code']])} of {len(dataset)} ({'{:.2%}'.format(len(bad[~bad['fixable with country code']])/len(dataset))})")
            display(bad[~bad['fixable with country code']].head(5))

        
        
        """for column in dataset.columns:
            if column in validation_dict:
                bad = validity_frame[~validity_frame[column].apply(validation_dict[column])][column]
                print(f"\ndropped {len(bad)} from {column}:")
                display(bad.head())
                validity_frame = validity_frame[validity_frame[column].apply(validation_dict[column])]"""

        """print(f"{dataset_name} original length: {len(dataset)}\nDROPPED: {len(dataset)-len(validity_frame)}")"""
        print("\n-------------------------------------------------------------")


### Uniqueness

Next let's take a look at whether or not there are any duplicate records in each dataset. We'll need a function for this...

In [ ]:
# We need a function to turn up suspected duplicates for stewarding
def find_dupes(frame: pd.DataFrame, pk_col: str, first_column: str, second_column: Optional[str]=None, ignore_originals: Optional[bool]=True) -> pd.DataFrame:
    """Creates a dataframe containing rows that are suspected duplicate records.
    Adds an extra column listing which other row they are a suspected dupe of.

    Args:
        frame: The dataframe to scan for duplicates
        pk_col: A column containing a pk-like value for this dataset
        first_column: The column on which to compare records.
        second_column: An optional second column to compare on. 
        ignore_originals: Optionally ignore the first instance in a set of duplicated rows

    Returns:
        A pandas dataframe containing all identified duplicates, flagged with the primary key of they row they duplicate
    """
    # turning off warnings because pandas doesn't know what's good for it.
    with pd.option_context('mode.chained_assignment', None):
        
        # skip rows that are null 
        notnullframe = frame[~frame[first_column].isna()]
        if not second_column is None:
            notnullframe = notnullframe[~notnullframe[second_column].isna()]

        # need to make a copy so we don't mess with the original
        temp_frame = notnullframe.copy(deep=True)
        
        # need a column to record why the dupe was flagged
        temp_frame["DUPLICATE_OF"] = np.nan
        
        # Add a temp column to hold our duplicate criterion
        if not second_column is None:       
            temp_frame[f"dupedetector"] = temp_frame[first_column].map(str) + temp_frame[second_column].map(str)
        else:
            temp_frame[f"dupedetector"] = temp_frame[first_column]
        
        # get the dupes
        dupes = temp_frame[temp_frame.duplicated(subset=f"dupedetector", keep=False)]

        # flag each dupe with its first instance
        for i, g in dupes.groupby(f"dupedetector"):
            for index, row in g.iterrows():
                dupes.loc[index, f"DUPLICATE_OF"] = g[pk_col].iloc[0]

        # delete that temporary column
        dupes = dupes.drop(f"dupedetector", axis=1)

        # optionally drop any row that is registered as a duplicate of itself
        if ignore_originals:
            dupes = dupes[dupes[f"DUPLICATE_OF"]!=dupes[pk_col]]  
        
        # return the dupes
        return dupes

Great, now we can analyse uniqueness and filter any potential duplicates out of the sources ahead of a test merge.

In [ ]:
for source, dataset in input_data.items():
    if 'PER_EMAIL_ADDR' in dataset.columns:
        input_data[source] = dataset.rename(columns={"PER_EMAIL_ADDR": 'PERSONAL_EMAIL'})


In [ ]:
# Need somewhere to store the duplicates we detect
all_dupes = dict()

# Iterate through each source, collecting duplicates based on each duplicate detection criterion
for source, dataset in input_data.items():
    all_dupes[source] = dict()
    for col1, col2 in DUPLICATE_CRITERIA:
        all_dupes[source][f"{col1}+{col2}"] = find_dupes(input_data[source], PK_FIELDS[source], col1, col2, True)

# For each source, collect all the duplicates and group them together
# If any were flagged twice, drop the double-ups
for source, dupedict in all_dupes.items():
    all_dupes[source]["all"] = pd.concat([dupeset for dupeset in dupedict.values()]).drop_duplicates(subset=PK_FIELDS[source], keep="first")
    
    # Uncomment the below for a sample of the flagged duplicates:
    # print(f"\n\n{source}")
    # display(all_dupes[source]["all"].sort_values("DUPLICATE_OF").head(6))

# We want to display the uniqueness for each source
to_display = pd.DataFrame({
    "source": [sourcename for sourcename in input_data.keys()],
    "length": [len(dataset) for dataset in input_data.values()],
    "flagged as duplicate": [len(all_dupes[source]["all"]) for source in [sourcename for sourcename in input_data.keys()]] 
})
to_display["uniqueness"] = 1 - to_display["flagged as duplicate"]/to_display["length"]

display(to_display.style.format({
            # we apply a little styling to our temp dataframe so that the validity column is displayed as a percentage
            "uniqueness": '{:,.2%}'.format
        }))

Uniqueness looks good, let's filter out those dupes

In [ ]:
data_to_merge = {
    # We can just concatenate the list of duplicates with the full dataset, then drop any rows that have a duplicate primary key
    # We also need to drop the "DUPLICATE_OF" column
    sourcename: pd.concat([duped_dataset, all_dupes[sourcename]["all"]]).drop_duplicates(subset=PK_FIELDS[sourcename], keep=False).drop(columns="DUPLICATE_OF") for sourcename, duped_dataset in input_data.items()
}


---

## Merging

Next we'll need to merge the data into a "Master Person" analogue. Up until now we have been using `pandas.Dataframe` objects to perform our analysis, but since we have a lot of data and need to do some complex joins, dataframes will be too computationally expensive moving forward. Instead we will convert our data to hash tables.

### Hash Tables

First let's create some convenient tools...

In [ ]:
def convert_to_dict(framename: str, frame: pd.DataFrame, keyfield: str) -> dict:
    """Converts a dataframe to a hash table. 
    
    Choose a field with 100% uniqueness and completeness for the keyfield.
    
    Args:
        framename: The name of the dataset being converted.
        frame: A pandas dataframe object to be converted.
        keyfield: The name of the column in the dataframe which should be used as they keys for the hash table. 

    Returns: 
        A dictionary.    
    """
    # Since we are using the keyfield as a hashtable key, we need to cut out any rows for which it is null.
    filled_rows = frame[frame[keyfield].notna()]

    # Similarly, we need every value for our keyfield to be unique.
    nodupes = filled_rows.drop_duplicates(subset=keyfield)

    # We are going to manipulate the dataframe in order to convert it, so let's make a copy to preserve the original.
    nodupes = nodupes.copy(deep=True)

    # We want to keep the original keyfield in each row, so let's copy it before we make it the dataframe index
    nodupes["newindex"]=nodupes[keyfield]
    nodupes.set_index("newindex", inplace=True)

    # Here we perform the conversion
    resultdict = nodupes.to_dict(orient="index")
    
    # A bit of telemetry here:
    if len(resultdict)>0:
        # I know it sounds ridiculous to say "primary-key duplicates"
        # Remember - we are dealing with static csv files, and just *treating* a field *like* a primary key!
        print(f"Converted {framename} to dict of length {len(resultdict)} based on {keyfield}, discarding {len(filled_rows)-len(nodupes)} primary-key duplicates in the process.")
    else:
        print(f"No suitable rows in {framename} to convert to dict based on {keyfield}.")
    
    return resultdict



Next, we can convert our data:

In [ ]:
# Need somewhere to store our hash tables
data_dicts = dict()

# We loop through each dataset
for dataset_name, dataset in data_to_merge.items():
    data_dicts[dataset_name] = convert_to_dict(
        framename=dataset_name, 
        frame=dataset, 
        # PK_FIELDS is one of our constants from the settings section above
        keyfield=PK_FIELDS[dataset_name]
    )

### 

### Finding matched records between datasets

Next we need a way to figure out when two rows from two different datasets refer to the same person. If we had QUT_IDENTITY_IDs for every dataset this would be a piece of cake, but unfortunately we do not. Instead, we need a workaround.

First, let's write a function to identify matched pairs of rows in two datasets, based on some field:

In [ ]:
def find_matches_as_tuples(matchfieldname: str, leftdict: dict, rightdict: dict, leftpk: str, rightpk: str) -> list[tuple[str]]:
    """Identifies entries in a pair of hash tables that appear to correspond to one another based on some field.

        NB: matchfield must not be heavily duplicated in input tables. filter them down if necessary
    Args: 
        matchfieldname: the name of the field on which to align the tables
        leftdict: the first table (analagous to the left element of an innner join)
        rightdict: the other table
        leftpk: the field in leftdict which acts as a primary key
        rightpk: the field in rightdict which acts as a primary key

    Returns:
        A list of tuples of the form (leftpk, rightpk), indicating a linked pair of entries from the two tables
    """
    # We reorient each input table to use the matchfield as its keys, ignoring entries with null keys
    # Note that this means if there are multiple rows with the same matchfield value in a table, only the last will count
    # For this reason it's a good idea to pick matchfields that are highly unique
    newleftdict = {element[matchfieldname]: element for element in leftdict.values() if not element[matchfieldname] is np.nan}
    newrightdict = {element[matchfieldname]: element for element in rightdict.values() if not element[matchfieldname] is np.nan}

    # This is hard to read, I know. 
    # We are iterating through each element in newleftdict, and checking if there's an element in newrightdict with the same value for matchfield
    # If there is, we record their pkfield values, if not we continue
    matches = [(leftelement[leftpk], newrightdict[matchfield][rightpk]) for matchfield, leftelement in newleftdict.items() if matchfield in newrightdict]

    # Telemetry
    print(f"Found {len(matches)} matches based on {matchfieldname}.")
    
    return matches

Next we'll collect those links! It may take a bit of trial and error to find the optimal fields for linking. The code below gets it right for the original input data. If your data has changed, YMMV. If all of your datasets have QUT_IDENTITY_ID as a column, you can simplify this process significantly.

In [ ]:
# A place to store our lists of links
linked_entries = dict()

# We're using the MATCHFIELDS variable we set up in the settings file to do this in a few lines of code
for matchfieldname, dataset_tuples in MATCHFIELDS.items():
    for leftset, rightset in dataset_tuples:

        # A bit of context for the telemetry
        print(f"\n{leftset} + {rightset}: \n\t", end="")

        # Actually doing the work now
        linked_entries[f"{leftset}__{rightset}"] = find_matches_as_tuples(
            matchfieldname=matchfieldname, 
            leftdict=data_dicts[leftset], 
            rightdict=data_dicts[rightset], 
            leftpk=PK_FIELDS[leftset], 
            rightpk=PK_FIELDS[rightset]
        )




### Agreement

Since we now know which records between sources correspond, we can look at how often those sources agree with each other on a given field's value.

First, you guessed it, we need another function:

In [ ]:
def evaluate_field_match(leftsourcedict: dict, rightsourcedict: dict, matchedrows: list[tuple[str]], fieldname: str) -> dict:
    """Evaluates the extent to which datasets with linked rows agree on the values of the fields in those rows.

    Args:
        leftsourcedict: a hash table containing entries, some of which are linked
        rightsourcedict: a hash table containing entries, some of which are those linked to in leftsourcedict
        matchedrows: a list of tuples of the form (leftsourcedict_pk, rightsourcedict_pk), indicating which rows are linked
        fieldname: the name of the field to compare values on

    Returns:
        A dict of the form {"agreeing": some_int, "compared" some_int}
    """
    # Another real eyeful I'm afraid.
    # We are making a list of booleans, representing whether or not the values in each linked pair of entries agree for a given field
    # Crucially, if one of the values is null, we ignore the pair
    evaluations = [leftsourcedict[pair[0]][fieldname]==rightsourcedict[pair[1]][fieldname] for pair in matchedrows if not (leftsourcedict[pair[0]][fieldname] is np.nan or rightsourcedict[pair[1]][fieldname] is np.nan)]

    # Then we return a dictionary representing the results
    return {
        "agreeing": len([x for x in evaluations if x is True]),
        "compared": len(evaluations)
    }


Now we can run the analysis:

In [ ]:
for pair, links in linked_entries.items():
    # As we iterate through the collections of linked entries, we need to know which datasets they're from
    # We can get that from the keys of our dict:
    leftsourcedict, rightsourcedict = pair.split("__")
    
    # We can only evaluate agreement on columns that appear in both datasets, so lets get a list of those:
    columns = [column for column in validation_dict.keys() if column in data_to_merge[leftsourcedict].columns and column in data_to_merge[rightsourcedict].columns]
    
    # We're running our function twice here which is inefficient, but the datasets aren't big enough for that to matter
    to_display = pd.DataFrame({
        "columns": columns,
        "compared": [evaluate_field_match(data_dicts[leftsourcedict], data_dicts[rightsourcedict], links, column)["compared"] for column in columns],
        "agreeing": [evaluate_field_match(data_dicts[leftsourcedict], data_dicts[rightsourcedict], links, column)["agreeing"] for column in columns], 
    })

    to_display["agreement"] = to_display["agreeing"]/to_display["compared"]

    print(f"\n\n{pair}: ")
    display(to_display.style.format({
            "agreement": '{:,.2%}'.format
        }))

Clearly there are a couple here where the sample size is too small to be considered.

### The Merge

The linked entries we calculated earlier look something like this, accounting for a little wiggle-room in the numbers. (The technique has been refined a little since this graphic was created.)

![Merge links topology](./img/merge_links.drawio.png)

In [ ]:
# Let's quickly put together a list of all the fields in all our datasets
all_fields =  set()
for dataset in data_to_merge.values():
    all_fields = all_fields | set(dataset.columns)

# We are going to create one giant hash-table, with an entry for each mastered person, and a sub-hash-table to record each system's value for each field
# To do this, we need a function that generates that sub-hash-table-tree
def def_value():
    fields = {field: dict() for field in all_fields}
    # We're also throwing in some information about personae here
    fields.update({
        "is_staff": False, 
        "is_applicant": False,  
        "is_student": False, 
        "is_prospect": False
    })
    return fields

# This is our one giant hash table
master_persons = defaultdict(def_value)

Here's an example of what one entry in our putative master person merge might look like. 
(This data is based on a real entry, but all values have been either redacted or modified. The individual was both a Studylink applicant and a SAMS student, and happened to have a mononym!):

```
'SOME-QUT-IDENTITY-ID-xxxxxxxxxxxx': {
    'APPLICANT_ID': {'studylink_applicants': '6257xxx'},
    'SUBRB_HOME': {'sams_students': 'Albion'},
    'STATE_HOME': {'sams_students': 'QLD'},
    'ADDR2_HOME': {'sams_students': '1 Mayfair'},
    'STUDENT_ID': {
        'sams_students': '1110xxxx',
        'studylink_applicants': '1110xxxx'
    },
    'CONTACT_ID': {},
    'FAMILY_NAME': {
        'sams_students': '-',
        'studylink_applicants': 'veronica'},
    'DOB': {
        'sams_students': '1993-xx-xx', 
        'studylink_applicants': '1993-xx-xx'
    },
    'SALUTATION': {
        'sams_students': 'mrs', 
        'studylink_applicants': 'mrs'
    },
    'ADDR3_HOME': {'sams_students': nan},
    'GIVEN_NAME': {
        'sams_students': 'veronica',
        'studylink_applicants': nan
    },
    'OTHER_NAME': {'sams_students': nan},
    'GENDER': {
        'sams_students': 'F', 
        'studylink_applicants': 'F'
    },
    'EMPLOYEE_ID': {},
    'PREFERRED_NAME': {
        'sams_students': 'veronica',
        'studylink_applicants': nan
    },
    'PHONE_MOBILE': {
        'sams_students': '614564xxxxx',
        'studylink_applicants': '614564xxxxx'
    },
    'PERSONAL_EMAIL': {
        'sams_students': 'veronica08@gmail.com',
        'studylink_applicants': 'myemail1993@gmail.com'
    },
    'FULL_NAME': {},
    'CNTRY_HOME': {
        'sams_students': 'AUS', 
        'studylink_applicants': 'AUSTRALIA'
    },
    'ADDR1_HOME': {'sams_students': 'Unit 999'},
    'POSTC_HOME': {'sams_students': '4007'},
    'is_staff': False,
    'is_applicant': True,
    'is_student': True,
    'is_prospect': False
}
```

In [ ]:
# A bit of delicate surgery here:

# First we add an entry for every SAMS student. Since we used the QUT_IDENTITY_ID field as the keys for this hash table, this is an easy 1-1 mapping.  
for index, row in data_dicts["sams_students"].items():
    for field, value in row.items():
        master_persons[index][field]["sams_students"] = value
    master_persons[index]["is_student"] = True

# Ditto for the SAMS applicants. Note that there are a handful of entries that are in both tables, and for these entries we now have multiple values for most fields
for index, row in data_dicts["sams_applicants"].items():
    for field, value in row.items():
        master_persons[index][field]["sams_applicants"] = value
    master_persons[index]["is_applicant"] = True

# Ascender employees aren't keyed by QUT_IDENTITY_ID, but we can pull out the matches by looking at the links we calculated earlier, and update the existing entries based on them.
for matchpair in linked_entries["ascender_employees__sams_students"]:
    # We are popping rows out here because we need to hang on to the unlinked entries in order to insert them later
    row = data_dicts["ascender_employees"].pop(matchpair[0])
    for field, value in row.items():
        master_persons[matchpair[1]][field]["ascender_employees"] = value
    master_persons[matchpair[1]]["is_staff"] = True

# Once we've iterated through the links between employees and SAMS students, we can do the same for employees vs SAMS applicants
for matchpair in linked_entries["ascender_employees__sams_applicants"]:
    # This time we need to check that the linked entry isn't one we already popped out because it was linked to a SAMS student
    if matchpair[0] in data_dicts["ascender_employees"]:
        row = data_dicts["ascender_employees"].pop(matchpair[0])
        for field, value in row.items():
            master_persons[matchpair[1]][field]["ascender_employees"] = value
        master_persons[matchpair[1]]["is_staff"] = True

# We can do the same for studylink applicants that are linked to SAMS students now.
for matchpair in linked_entries["sams_students__studylink_applicants"]:
    row = data_dicts["studylink_applicants"].pop(matchpair[1])
    for field, value in row.items():
        master_persons[matchpair[0]][field]["studylink_applicants"] = value
    master_persons[matchpair[0]]["is_applicant"] = True

# Same for the studylink applicants that are linked to SAMS applicants
for matchpair in linked_entries["studylink_applicants__sams_applicants"]:
    # once again, we need to check that the linked entry isn't one we already popped out because it was linked to a SAMS student
    if matchpair[0] in data_dicts["studylink_applicants"]:
        row = data_dicts["studylink_applicants"].pop(matchpair[0])
        for field, value in row.items():
            master_persons[matchpair[1]][field]["studylink_applicants"] = value
        master_persons[matchpair[0]]["is_applicant"] = True

Quick pause to assess where we are up to in our surgery:

We've rolled in ALL of the records from SAMS, as well as any records we can link to them from Ascender and Studylink.

Taking a look at the merge link topology diagram above, we can see that there is only one remaining link so account for - the one between DEP and Ascender. However, since we haven't rolled in all of our Ascender records yet, there's no guarantee that we can bring that final link in. Let's get all the other datasets fully folded in first before we try to add in that last link.

Up until now we have only dealt with records that we knew had a value for QUT_IDENTITY_ID. Since our Master Person dataset is keyed based on QUT_IDENTITY_ID, we need to make sure that all the remaining records have a value for that field. Where the value is missing, we will forge a fake ID for them.

In [ ]:
count = 0

for source in ["ascender_employees", "studylink_applicants", "dep_prospects"]:
    for row, fields in data_dicts[source].items():
        if ("QUT_GUID" in fields and fields["QUT_GUID"] is np.nan) or not "QUT_GUID" in fields:
            data_dicts[source][row]["QUT_GUID"] = f"NOID_{count}"
            count +=1 

Now we can roll in the remaining ascender and studylink records:

In [ ]:
for row, fields in data_dicts["ascender_employees"].items():
    for field, value in fields.items():
        master_persons[fields["QUT_GUID"]][field]["ascender_employees"] = value
    master_persons[fields["QUT_GUID"]]["is_staff"] = True

for row, fields in data_dicts["studylink_applicants"].items():
    for field, value in fields.items():
        master_persons[fields["QUT_GUID"]][field]["studylink_applicants"] = value
    master_persons[fields["QUT_GUID"]]["is_applicant"] = True

Finally we can link in that pesky DEP record:

In [ ]:
for matchpair in linked_entries["dep_prospects__ascender_employees"]:
    row = data_dicts["dep_prospects"].pop(matchpair[0])
    for field, value in row.items():
        master_persons[matchpair[1]][field]["dep_prospects"] = value
    master_persons[matchpair[1]]["is_prospect"] = True

And then the rest of the DEP data

In [ ]:
for row, fields in data_dicts["dep_prospects"].items():
    for field, value in fields.items():
        master_persons[fields["QUT_GUID"]][field]["dep_prospects"] = value
    master_persons[fields["QUT_GUID"]]["is_prospect"] = True

We did it! Let's take a look at what we have...

In [ ]:
print(f"staff: {len([x for x in master_persons.values() if x['is_staff']])}")
print(f"students: {len([x for x in master_persons.values() if x['is_student']])}")
print(f"applicants: {len([x for x in master_persons.values() if x['is_applicant']])}")
print(f"prospects: {len([x for x in master_persons.values() if x['is_prospect']])}")
print("-----")
print(f"people who are both staff AND student: {len([x for x in master_persons.values() if (x['is_staff'] and x['is_student'])])}")
print(f"people who are both staff AND applicant: {len([x for x in master_persons.values() if (x['is_staff'] and x['is_applicant'])])}")
print(f"people who are both student AND applicant: {len([x for x in master_persons.values() if (x['is_applicant'] and x['is_student'])])}")
print("-----")
print(f"people who are staff AND applicant AND staff: {len([x for x in master_persons.values() if (x['is_applicant'] and x['is_student'] and x['is_staff'])])}")

### Consistency

In [ ]:
# Not every person who has an entry in more than one system has more than one value for every field (some fields are null). 
# So how many of each field has multiple values recorded against it?
def returns_0():
    return 0

multi_value_counts = defaultdict(returns_0)

for row, fields in master_persons.items():
    for field, values in fields.items():
        if isinstance(values, dict) and len(list(filter(lambda y: not y is np.nan, values.values()))) > 1:
            multi_value_counts[field] += 1


# For those people who had entries in more than one system, how many of them had different values for their fields in different systems?
differing_value_counts = defaultdict(returns_0)

for row, fields in master_persons.items():
    for field, values in fields.items():
        if isinstance(values, dict) and len(set(filter(lambda y: not y is np.nan, values.values()))) > 1:
            differing_value_counts[field] += 1


# We can use that info to calculate consistency for the master person 
# (for the individual systems we didn't have a good way to measure this, so we used agreement as a proxy measure)
consistency_frame = pd.DataFrame({
    "fields": [field for field in FIELD_SYSTEM_PRECEDENCES],
    "entries with multiple values": [multi_value_counts[field] if field in multi_value_counts else 0 for field in FIELD_SYSTEM_PRECEDENCES],
    "entries with inconsistent values": [differing_value_counts[field] if field in differing_value_counts else 0 for field in FIELD_SYSTEM_PRECEDENCES]
})
consistency_frame["consistency"] = 1 - consistency_frame["entries with inconsistent values"]/consistency_frame["entries with multiple values"]

display(consistency_frame.style.format({
            # we apply a little styling to our temp dataframe so that the validity column is displayed as a percentage
            "consistency": '{:,.2%}'.format
        }))

Just out of interest, let's take a look at how many people have multiple different VALID values for each field. Note that we aren't validating addresses, so they're not included here.

In [ ]:
differing_valid_value_counts = defaultdict(returns_0)

for row, fields in master_persons.items():
    for field, values in fields.items():
        if isinstance(values, dict) and field in validation_dict and len(set(filter(lambda y: not y is np.nan and (validation_dict[field](y)), values.values()))) > 1:
            differing_valid_value_counts[field] += 1

for field, count in differing_valid_value_counts.items():
    print(f"{field}: {count}")

### Collapsing the field possibilities

Where we have more than one possible value for a field given to us by different systems, we want to pick the best value. We'll be using the FIELD_SYSTEM_PREFERENCES we set down in the settings section. These are based on the agreement measures we calculated earlier - the higher a system's overall agreement, the higher its priority. Let's write a function to do that:

In [ ]:
def choose_value(field_name, field_values):
    # If this isn't a dict of possible values, just return it as-is 
    if not isinstance(field_values, dict):
        return field_values

    # If we haven't listed a preference order for this field, we don't need it in the master person, so return null    
    if not field in FIELD_SYSTEM_PRECEDENCES:
        return np.nan

    # Grab all the non-null values
    non_null_values = {source: value for source, value in field_values.items() if not value is np.nan}

    # If there are no non-null values in the options, set the field to null
    if not non_null_values:
        return np.nan

    # If this is a field we don't have a way of validating, return the preferred source's non-null value
    if not field_name in validation_dict:
        for source in FIELD_SYSTEM_PRECEDENCES[field_name]:
            if source in non_null_values:
                return non_null_values[source]

    # Now grab all the valid values
    valid_values = {source: value for source, value in non_null_values.items() if validation_dict[field_name](value)}

    # If there are no valid values, take the preferred source's invalid value
    if not valid_values:
        for source in FIELD_SYSTEM_PRECEDENCES[field_name]:
            if source in non_null_values:
                return non_null_values[source]

    # Finally, if you got this far, take the preferred source's valid value
    for source in FIELD_SYSTEM_PRECEDENCES[field_name]:
        if source in valid_values:
            return valid_values[source]


# Let's use that function to collapse the fields:
final_master_persons = dict()

for id, person in master_persons.items():
    final_master_persons[id] = dict()
    for field, values in person.items():
        final_master_persons[id][field] = choose_value(field, values)
        final_master_persons[id]["QUT_GUID"] = id

Now that we have just one value per field for each record, we can convert our master persons back to a dataframe.

In [ ]:
master_frame = pd.DataFrame.from_dict(final_master_persons, orient='index').drop(columns=['APPLICANT_ID', 'STUDENT_ID', 'CONTACT_ID', 'EMPLOYEE_ID'])
master_frame.to_csv("master_frame.csv")

### Master Person Completeness

Let's calculate the completeness for the master person data:

In [ ]:
dataset_length = len(master_frame)

# Then we create a temporary dataframe to hold our calculations
to_display = pd.DataFrame({
    # The first column of the temp dataframe contains the names of each column in the dataset
    "column": [column for column in master_frame.columns],
    # The second column shows how many non-null values there are for each column
    "count": [len(master_frame[master_frame[column].notna()]) for column in master_frame.columns],
    # The third column calculates the relative frequency of non-null values 
    "completeness": [len(master_frame[master_frame[column].notna()])/dataset_length for column in master_frame.columns],
    # The fourth column gives us an example value from the column
    "sample": [master_frame[master_frame[column].notna()][column].mode()[0] if len(master_frame[master_frame[column].notna()][column]) else np.nan for column in master_frame]
# we apply a little styling to our temp dataframe so that the completeness column is displayed as a percentage
}).style.format({
    "completeness": '{:,.2%}'.format
})

# This display() function is the one we imported from Ipython up the top - it just improves readability
display(to_display)

### Master Person Validity

Same for validity:

In [ ]:
# first, we need one temp frame to hold the results of our validations
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=UserWarning)
    validity_frame = pd.DataFrame()

    # our validity_frame mirrors the
    for column in master_frame.columns:
        if column in validation_dict:
            validity_frame[column] = master_frame[column].apply(validation_dict[column])    

    to_display = pd.DataFrame({
        # The first column of the temp dataframe contains the names of each column in the dataset
        "column": [column for column in validity_frame.columns],
        # The second column shows how many non-null values there are for each column
        "count": [len(master_frame[master_frame[column].notna()]) for column in validity_frame.columns],
        # The third column calculates how many valid values there are in each column 
        "valid": [len(validity_frame[validity_frame[column].notna()][validity_frame[column]==True]) for column in validity_frame.columns],
    
    })

    # Finally let's add another column to show validity as a percentage
    to_display["validity"] = to_display["valid"]/to_display["count"]

    display(to_display.style.format({
        # we apply a little styling to our temp dataframe so that the validity column is displayed as a percentage
        "validity": '{:,.2%}'.format
    }))

### Master Person Uniqueness

Finally, we can take a look at uniqueness for the master person records. This also gives us an idea of the stewardship requirements that our master person system will have:

In [ ]:
mdh_frame_copy = master_frame.copy(deep=True)

mobilename_quarantine=find_dupes(mdh_frame_copy, "QUT_GUID", "GIVEN_NAME", "PHONE_MOBILE")
print(f"Quarantined {len(mobilename_quarantine)} rows based on mobile + firstname")
mobiledob_quarantine=find_dupes(mdh_frame_copy, "QUT_GUID", "DOB", "PHONE_MOBILE")
print(f"Quarantined {len(mobiledob_quarantine)} rows based on mobile + dob")
emailname_quarantine=find_dupes(mdh_frame_copy, "QUT_GUID", "GIVEN_NAME", "PERSONAL_EMAIL")
print(f"Quarantined {len(emailname_quarantine)} rows based on email + firstname")
emaildob_quarantine=find_dupes(mdh_frame_copy, "QUT_GUID", "DOB", "PERSONAL_EMAIL")
print(f"Quarantined {len(emaildob_quarantine)} rows based on email + dob")

In [ ]:
all_master_dupes = pd.concat([mobilename_quarantine, mobiledob_quarantine, emailname_quarantine, emaildob_quarantine]).drop_duplicates(subset="QUT_GUID", keep='first')
print(f"Total records quarantined: {len(all_master_dupes)}")

In [ ]:
print(f"Master person uniqueness: {'{0:.2%}'.format(1- len(all_master_dupes)/len(mdh_frame_copy))}")